In [1]:
import os
import sys
import glob
import random
import torchvision
import torch
import cv2
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import timm

from albumentations import *
from albumentations.pytorch import ToTensorV2
import torch
import torch.utils.data as data
from PIL import Image
from adamp import AdamP
from adamp import SGDP
from tqdm.notebook import tqdm
from time import time
import matplotlib.pyplot as plt

# Dataset, Transformer 정의

In [2]:
from albumentations import *
from albumentations.pytorch import ToTensorV2
def get_transforms(need=('train', 'val'), img_size=(512, 384), mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
    """
    train 혹은 validation의 augmentation 함수를 정의합니다. train은 데이터에 많은 변형을 주어야하지만, validation에는 최소한의 전처리만 주어져야합니다.
    
    Args:
        need: 'train', 혹은 'val' 혹은 둘 다에 대한 augmentation 함수를 얻을 건지에 대한 옵션입니다.
        img_size: Augmentation 이후 얻을 이미지 사이즈입니다.
        mean: 이미지를 Normalize할 때 사용될 RGB 평균값입니다.
        std: 이미지를 Normalize할 때 사용될 RGB 표준편차입니다.

    Returns:
        transformations: Augmentation 함수들이 저장된 dictionary 입니다. transformations['train']은 train 데이터에 대한 augmentation 함수가 있습니다.
    """
    transformations = {}
    if 'train' in need:
        transformations['train'] = Compose([
            #Resize(img_size[0], img_size[1], p=1.0),
            #Resize(200, 260, p=1.0),
            CenterCrop(height = 400, width = 200), # add centercrop 350/350 -> 400/200 -> 300/300
            #HorizontalFlip(p=0.5),
            #ShiftScaleRotate(p=0.5),
            #HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            #GaussNoise(p=0.5),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    if 'val' in need:
        transformations['val'] = Compose([
            #Resize(img_size[0], img_size[1]),
            #Resize(200, 260),
            CenterCrop(height = 400, width = 200), # add centercrop
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    return transformations

In [3]:
class MaskLabels:
    mask = 0
    incorrect = 1
    normal = 2

class GenderLabels:
    male = 0
    female = 1

class AgeGroup:
    map_label = lambda x: 0 if int(x) < 30 else 1 if int(x) < 58 else 2

In [4]:
class MaskBaseDataset(data.Dataset):
    num_classes = 3 * 2 * 3
    
#     IMG_EXTENSIONS = [
#         ".jpg", ".JPG", ".jpeg", ".JPEG", ".png",
#         ".PNG", ".ppm", ".PPM", ".bmp", ".BMP",
#     ]
    
    _file_names = {
        "mask1": MaskLabels.mask,
        "mask2": MaskLabels.mask,
        "mask3": MaskLabels.mask,
        "mask4": MaskLabels.mask,
        "mask5": MaskLabels.mask,
        "incorrect_mask": MaskLabels.incorrect,
        "normal": MaskLabels.normal
    }

    image_paths = []
    mask_labels = []
    gender_labels = []
    age_labels = []

    def __init__(self, img_dir, mean, std, transform=None):
        """
        MaskBaseDataset을 initialize 합니다.

        Args:
            img_dir: 학습 이미지 폴더의 root directory 입니다.
            transform: Augmentation을 하는 함수입니다.
        """
        self.img_dir = img_dir
        self.mean = mean
        self.std = std
        self.transform = transform
#         self.val_transform = transform

        self.setup()

    def set_transform(self, transform):
        """
        transform 함수를 설정하는 함수입니다.
        """
        self.transform = transform

#     def set_val_transform(self, transform):
#         """
#         transform 함수를 설정하는 함수입니다.
#         """
#         self.val_transform = transform        
        
    def setup(self):
        """
        image의 경로와 각 이미지들의 label을 계산하여 저장해두는 함수입니다.
        """
        profiles = os.listdir(self.img_dir)
        for profile in profiles:
            
            if profile.startswith("."):  # "." 로 시작하는 파일은 무시합니다
                continue
                
            img_folder = os.path.join(self.img_dir, profile)
            
            for file_name in os.listdir(img_folder):
                _file_name, ext = os.path.splitext(file_name)
                if _file_name not in self._file_names:  # "." 로 시작하는 파일 및 invalid 한 파일들은 무시합니다
                    continue
                    
                img_path = os.path.join(self.img_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
                if os.path.exists(img_path):
                    self.image_paths.append(img_path)
                    mask_label = self._file_names[_file_name]
                    self.mask_labels.append(mask_label)

                    id, gender, race, age = profile.split("_")
                    gender_label = getattr(GenderLabels, gender)
                    age_label = AgeGroup.map_label(age)

                    self.gender_labels.append(gender_label)
                    self.age_labels.append(age_label)

    def __getitem__(self, index):
        """
        데이터를 불러오는 함수입니다. 
        데이터셋 class에 데이터 정보가 저장되어 있고, index를 통해 해당 위치에 있는 데이터 정보를 불러옵니다.
        
        Args:
            index: 불러올 데이터의 인덱스값입니다.
        """
        # 이미지를 불러옵니다.
        image_path = self.image_paths[index]
        image = Image.open(image_path)
        
        # 레이블을 불러옵니다.
        mask_label = self.mask_labels[index]
        gender_label = self.gender_labels[index]
        age_label = self.age_labels[index]
        multi_class_label = mask_label * 6 + gender_label * 3 + age_label
        
        # 이미지를 Augmentation 시킵니다.
        if self.transform:
            image = self.transform(image=np.array(image))['image']
        return image, multi_class_label

    def __len__(self):
        return len(self.image_paths)

## Train Dataset

In [5]:
BASE_DIR = '/opt/ml/input/data/train'
IMG_DIR = f'{BASE_DIR}/images'
TRAIN_CSV = f'{BASE_DIR}/train.csv'
TRAIN_BATCH_SIZE = 32
VAL_BATCH_SIZE = 32

RGB_Mean = [0.56019358, 0.52410121, 0.501457]
RGB_SD = [0.23318603, 0.24300033, 0.24567522]

In [6]:
transform = get_transforms(mean=RGB_Mean, std=RGB_SD)
    
dataset = MaskBaseDataset(
    img_dir=IMG_DIR,
    mean = RGB_Mean,
    std = RGB_SD
)


In [7]:
next(iter(dataset))

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=384x512 at 0x7FB667644A00>,
 9)

# Loss

In [8]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )

In [9]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=3, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))


In [10]:
class F1Loss(nn.Module):
    def __init__(self, classes=3, epsilon=1e-7):
        super().__init__()
        self.classes = classes
        self.epsilon = epsilon
    def forward(self, y_pred, y_true):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.classes).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1 - self.epsilon)
        return 1 - f1.mean()


In [11]:
_criterion_entrypoints = {
    'cross_entropy': nn.CrossEntropyLoss,
    'focal': FocalLoss,
    'label_smoothing': LabelSmoothingLoss,
    'f1': F1Loss
}

def criterion_entrypoint(criterion_name):
    return _criterion_entrypoints[criterion_name]


def is_criterion(criterion_name):
    return criterion_name in _criterion_entrypoints


def create_criterion(criterion_name, **kwargs):
    if is_criterion(criterion_name):
        create_fn = criterion_entrypoint(criterion_name)
        criterion = create_fn(**kwargs)
    else:
        raise RuntimeError('Unknown loss (%s)' % criterion_name)
    return criterion


In [12]:
def f1_loss(y_true:torch.Tensor, y_pred:torch.Tensor, is_training=False) -> torch.Tensor:
    
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
        
    
    tp = (y_true * y_pred).sum().to(torch.float32)
    tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)
    
    epsilon = 1e-7
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2* (precision*recall) / (precision + recall + epsilon)
    f1.requires_grad = is_training
    return f1

In [13]:
class ArcFaceLoss(nn.modules.Module):
    def __init__(self, s=45.0, m=0.1, crit="bce", weight=None, reduction="mean"):
        super().__init__()

        self.weight = weight
        self.reduction = reduction
        
        if crit == "focal":
            self.crit = FocalLoss(gamma=args.focal_loss_gamma)
        elif crit == "bce":
            self.crit = nn.CrossEntropyLoss(reduction="none")   

        if s is None:
            self.s = torch.nn.Parameter(torch.tensor([45.], requires_grad=True, device='cuda'))
        else:
            self.s = s

        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m
        
    def forward(self, logits, labels):

        # logits = logits.float()
        cosine = logits
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        
        labels2 = torch.zeros_like(cosine)
        labels2.scatter_(1, labels.view(-1, 1).long(), 1)
        output = (labels2 * phi) + ((1.0 - labels2) * cosine)

        s = self.s

        output = output * s
        loss = self.crit(output, labels)

        if self.weight is not None:
            w = self.weight[labels].to(logits.device)

            loss = loss * w
            ### human coding
            class_weights_norm = 'batch'
            if class_weights_norm == "batch":
                loss = loss.sum() / w.sum()
            if class_weights_norm == "global":
                loss = loss.mean()
            else:
                loss = loss.mean()
            
            return loss
        if self.reduction == "mean":
            loss = loss.mean()
        elif self.reduction == "sum":
            loss = loss.sum()
        return loss

# Model

In [14]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('GPU Name:', torch.cuda.get_device_name(0))
else:
    print('No GPU, using CPU.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
GPU Name: Tesla V100-PCIE-32GB


In [15]:
# from efficientnet_pytorch import EfficientNet
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 18):
        super(MyModel, self).__init__()
        # Transfer learning to add final layers in the end.
        # Model Comaprison: https://paperswithcode.com/sota/image-classification-on-imagenet
        # self.backbone = models.resnet50(pretrained=True)
        # self.backbone = models.resnext50_32x4d(pretrained=True)
        
        model_architecture = 'tf_efficientnet_b4'
        self.backbone = timm.create_model(model_architecture, pretrained=True)
        n_features = self.backbone.classifier.in_features
        self.backbone.fc = nn.Linear(in_features=n_features, out_features=num_classes, bias=True)
        #self.backbone.fc = nn.Linear(in_features=2048, out_features=18, bias=True)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.backbone(x)
        return x

In [16]:
# torch.cuda.empty_cache()

In [17]:
model = MyModel()
model.cuda()

MyModel(
  (backbone): EfficientNet(
    (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_run

In [18]:
!nvidia-smi

Tue Aug 31 18:52:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:00:05.0 Off |                  Off |
| N/A   58C    P0   185W / 250W |  14756MiB / 32510MiB |     66%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Train

In [19]:
# torch.cuda.empty_cache()

EPOCHS = 3
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 1e-5

In [20]:
from adamp import AdamP

# # Freeze the feature extracting convolution layers
# for param in model.features.parameters():
#     param.requires_grad = False

criterion = FocalLoss(gamma = 5)
# criterion = ArcFaceLoss()
# criterion = CutMixCrossEntropyLoss(True)
# criterion = nn.CrossEntropyLoss()

#optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
optimizer = AdamP(model.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999), weight_decay= WEIGHT_DECAY)
#optimizer = SGDP(model.parameters(), lr=LEARNING_RATE, weight_decay= WEIGHT_DECAY,momentum=0.9, nesterov=True)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500,1000,1500], gamma=0.5)

In [21]:
class Metrics(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [24]:
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold , StratifiedKFold
import sys
sys.path.append("..")
import slack_noti
import copy

fold_count = 5

def train(model, epochs, dataset, save_path):
    best_valid_acc = 0
    best_valid_loss = 100000
    
#     kfold = KFold(n_splits=fold_count, random_state=0, shuffle=True)
    stratified_kfold = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
    labels = [y for _,y in dataset]
    
    k_idx=0
#     for train_index, validate_index in kfold.split(dataset):    # kfold
    for train_index, validate_index in stratified_kfold.split(dataset, labels):    #s kfold
        k_idx+=1
        print(f'####### K-FOLD :: {k_idx}th')
        slack_noti.noti(f'####### K-FOLD :: {k_idx}th')
        train_dataset = torch.utils.data.dataset.Subset(dataset,train_index)
        
        copied_dataset = copy.deepcopy(dataset)
        valid_dataset = torch.utils.data.dataset.Subset(copied_dataset,validate_index)
        
#         train_dataset.dataset.set_transform(transform['train'])
#         validate_dataset.dataset.set_transform(transform['val'])
        
        train_dataset.dataset.set_transform(transform['train'])
        valid_dataset.dataset.set_transform(transform['val'])        

        train_loader = data.DataLoader(
            train_dataset,
            batch_size=TRAIN_BATCH_SIZE, 
            num_workers=2,
            shuffle=True)

        valid_loader = data.DataLoader(
            valid_dataset,
            batch_size=VAL_BATCH_SIZE,
            num_workers=2,
            shuffle=False)   
            
        for epoch in range(epochs):
            epoch_train_loss, epoch_train_acc = Metrics(), Metrics()
            early_stop = 0
            for iter, (img,label) in enumerate(tqdm(train_loader)):
                optimizer.zero_grad()

                img, label = img.type(torch.FloatTensor).to(device), label.to(device)

                # 모델에 이미지 forward
                model.train()
                pred_logit = model(img)

                # loss 값 계산
                loss = criterion(pred_logit,label)

                # Backpropagation
                loss.backward()
                optimizer.step()
                scheduler.step()

                # Accuracy 계산
                pred_label = torch.max(pred_logit.data,1)

                # pred_label  = pred_logit.argmax(1)
                acc = (pred_label.indices == label).sum().item() / len(label)

                epoch_train_loss.update(loss.item(), len(img))
                epoch_train_acc.update(acc, len(img))

                train_loss = loss.item()
                train_acc = acc
                
    #             if (iter % 300 == 0) or (iter == len(train_loader)-1):
    #                 print("Iter [%3d/%3d] | Train Loss %.4f | Train Acc %.4f" %(iter, len(train_loader), train_loss, train_acc))

            epoch_train_loss = epoch_train_loss.avg
            epoch_train_acc = epoch_train_acc.avg
            print("Epoch %d | Train Loss %.4f | Train Acc %.4f"%(epoch,epoch_train_loss, epoch_train_acc))
            slack_noti.noti("Epoch %d | Train Loss %.4f | Train Acc %.4f"%(epoch,epoch_train_loss, epoch_train_acc))
            
            valid_loss, valid_acc, valid_f1 = Metrics(), Metrics(), Metrics()
            model.eval()
            
            for img, label in valid_loader:
                img, label = img.type(torch.FloatTensor).to(device), label.to(device)

                with torch.no_grad():
                    pred_logit = model(img)
                loss = criterion(pred_logit, label) 
                pred_label = torch.max(pred_logit.data,1)

                acc = (pred_label.indices == label).sum().item() / len(label)

                valid_loss.update(loss.item(), len(img))
                valid_acc.update(acc, len(img))
                valid_f1.update(f1_loss(label, pred_label.indices), len(img))


            valid_loss = valid_loss.avg
            valid_acc = valid_acc.avg
            valid_f1 = valid_f1.avg
            print("Valid Loss %.4f | Valid Acc %.4f | f1 score %.4f" %(valid_loss, valid_acc, valid_f1))
            slack_noti.noti("Valid Loss %.4f | Valid Acc %.4f | f1 score %.4f" %(valid_loss, valid_acc, valid_f1))
            
            if valid_loss <= best_valid_loss:
                print("New valid model for val loss! saving the model...")
                torch.save(model.state_dict(),PATH + f"v4_stkfold5_epoch3_transformer_{k_idx}th_{epoch:03}_loss_{valid_loss:4.2}.ckpt")
                best_valid_loss = valid_loss
                early_stop = 0
            else:
                early_stop += 1
                if early_stop > 2:
                    break
            
            if valid_acc > best_valid_acc:
                print("New valid model for val accuracy! saving the model...")
                torch.save(model.state_dict(),PATH + f"v4_stkfold5_epoch3_transformer_{k_idx}th_{epoch:03}_loss_{valid_loss:4.2}.ckpt")
                best_valid_acc = valid_acc
            print()

In [25]:
PATH = "./models/v4_stkfold5_epoch3_transformer/"

# (Train Image Size = 1890 * 9) / (Batch Size = 32) = 532 as Length
train(model, EPOCHS, dataset, PATH)

####### K-FOLD :: 1th



Epoch 0 | Train Loss 0.2493 | Train Acc 0.7864
Valid Loss 0.0666 | Valid Acc 0.8130 | f1 score 9.2831
New valid model for val loss! saving the model...
New valid model for val accuracy! saving the model...




Epoch 1 | Train Loss 0.0148 | Train Acc 0.9279
Valid Loss 0.0549 | Valid Acc 0.8397 | f1 score 9.3598
New valid model for val loss! saving the model...
New valid model for val accuracy! saving the model...




Epoch 2 | Train Loss 0.0053 | Train Acc 0.9654
Valid Loss 0.0606 | Valid Acc 0.8423 | f1 score 9.3488
New valid model for val accuracy! saving the model...

####### K-FOLD :: 2th



Epoch 0 | Train Loss 0.0123 | Train Acc 0.9438
Valid Loss 0.0021 | Valid Acc 0.9870 | f1 score 9.3717
New valid model for val loss! saving the model...
New valid model for val accuracy! saving the model...



...


Epoch 1 | Train Loss 0.0012 | Train Acc 0.9923
Valid Loss 0.0002 | Valid Acc 0.9995 | f1 score 9.3730
New valid model for val loss! saving the model...
New valid model for val accuracy! saving the model...




Epoch 2 | Train Loss 0.0008 | Train Acc 0.9960
Valid Loss 0.0002 | Valid Acc 0.9974 | f1 score 9.3710

####### K-FOLD :: 5th



Epoch 0 | Train Loss 0.0010 | Train Acc 0.9936
Valid Loss 0.0003 | Valid Acc 0.9968 | f1 score 9.3670




Epoch 1 | Train Loss 0.0013 | Train Acc 0.9947
Valid Loss 0.0007 | Valid Acc 0.9944 | f1 score 9.3700




Epoch 2 | Train Loss 0.0019 | Train Acc 0.9940
Valid Loss 0.0005 | Valid Acc 0.9971 | f1 score 9.3668



In [ ]:
# !nvidia-smi

# Infrence

In [26]:
from torch.utils.data import Dataset, DataLoader

class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image, self.img_paths[index]

    def __len__(self):
        return len(self.img_paths)

# class TestDataset(data.Dataset):
#     def __init__(self, img_paths, transform):
#         self.img_paths = img_paths
#         self.transform = transform

#     def __getitem__(self, index):
#         image = Image.open(self.img_paths[index])

#         if self.transform:
#             image = self.transform(image)
# #             image_transform = self.transform(image=np.array(image))['image']
#         return image

#     def __len__(self):
#         return len(self.img_paths)

In [27]:
model_path = "/opt/ml/code/efficietnet/models/v4_stkfold5_epoch3_transformer/v4_stkfold5_epoch3_transformer_2th_000_loss_0.0021.ckpt"
model = MyModel()
model.load_state_dict(torch.load(model_path))
model.cuda()

MyModel(
  (backbone): EfficientNet(
    (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_run

In [28]:
from torchvision import transforms
TEST_DIR = "/opt/ml/input/data/eval"

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(TEST_DIR, 'info.csv'))
image_dir = os.path.join(TEST_DIR, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
valid_transform = transforms.Compose([
#     Resize((512, 384), Image.BILINEAR),
    transforms.CenterCrop((400,200)),
    transforms.ToTensor(),
    transforms.Normalize((0.56019358, 0.52410121, 0.501457), (0.23318603, 0.24300033, 0.24567522)),
])
dataset = TestDataset(image_paths, valid_transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

device = torch.device('cuda')

model.eval()

pred_result = []
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images, path in tqdm(loader):
    temp = []
    temp.append(path)
    with torch.no_grad():
        images = images.type(torch.FloatTensor).to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        temp.append(pred)
        all_predictions.extend(pred.cpu().numpy())
    pred_result.append(temp)
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
PRETRAINED_MODEL_NAME = "eff_b4_stkkfold_v4"
file_name = f"{PRETRAINED_MODEL_NAME}_submission.csv"
submission.to_csv(os.path.join(TEST_DIR, file_name), index=False)
print('test inference is done!')


test inference is done!
